In [1]:
from head_detector import HeadDetector
import cv2
import os
import numpy as np

from typing import List, Tuple

/home/jocareher/anaconda3/envs/vgg_heads/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
detector = HeadDetector()
image_path = "/home/jocareher/Documents/baby_face_72/images/face_bcn_02.JPG"
predictions = detector(image_path)
# predictions.heads contain a list of heads with .bbox, .vertices_3d, .head_pose params
result_image = predictions.draw() # draw heads on the image
cv2.imwrite("result.png",result_image) # save result image to previe

True

In [6]:
def load_landmarks(file_path: str) -> List[Tuple[float, float]]:
    """
    Load landmarks from a label file.
    """
    # Verify if file exists and is not empty
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        print(f"[WARNING] Empty or missing file: {file_path}")
        return []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse each line into 2D coordinates
    landmarks = []
    for line in lines:
        try:
            coords = np.array(line.strip().split(), dtype=float).reshape(-1, 2)
            landmarks.extend(coords.tolist())
        except ValueError:
            print(f"[WARNING] Invalid line in file {file_path}: {line.strip()}")

    # Return an empty list if no valid landmarks were found
    if not landmarks:
        print(f"[WARNING] No valid landmarks found in file: {file_path}")
        return []

    return landmarks


def match_unique_landmarks(dlib_landmarks: np.ndarray, vgg_landmarks: np.ndarray, coord_output_file: str, index_output_file: str):
    """
    Guarda los puntos más cercanos en formato x1 y1 ... xn yn para las coordenadas y indices = [indx] para los índices.

    :param dlib_landmarks: np.ndarray de forma (68, 2), coordenadas de DLIB.
    :param vgg_landmarks: np.ndarray de forma (N, 2), coordenadas de VGG Heads.
    :param coord_output_file: Nombre del archivo donde se guardarán las coordenadas en una línea.
    :param index_output_file: Nombre del archivo donde se guardarán los índices en formato de lista.
    """
    matched_indices = []
    matched_landmarks = []
    used_indices = set()

    for dlib_point in dlib_landmarks[:68]:  # Asegurarse de procesar solo 68 puntos
        distances = np.linalg.norm(vgg_landmarks - dlib_point, axis=1)
        for idx in used_indices:
            distances[idx] = np.inf
        closest_index = np.argmin(distances)
        used_indices.add(closest_index)
        matched_indices.append(closest_index)
        matched_landmarks.append(vgg_landmarks[closest_index])

    # Guardar las coordenadas en una línea
    with open(coord_output_file, "w") as coord_file:
        flattened_landmarks = [f"{int(coord[0])} {int(coord[1])}" for coord in matched_landmarks]
        coord_file.write(" ".join(flattened_landmarks) + "\n")

    # Guardar los índices en formato de lista
    with open(index_output_file, "w") as index_file:
        index_file.write(f"indices = {matched_indices}\n")

    print(f"Coordenadas guardadas en {coord_output_file}")
    print(f"Índices guardados en {index_output_file}")

# Cargar los archivos de entrada
dlib_landmarks = np.loadtxt("test.txt").reshape(-1, 2)  # Archivo con 68 landmarks
vgg_landmarks = np.loadtxt("landmarks_2d.txt").reshape(-1, 2)  # Archivo con 2500+ landmarks

# Ejecutar el cálculo
match_unique_landmarks(dlib_landmarks, vgg_landmarks, "matched_coordinates.txt", "matched_indices.txt")


In [8]:
import os
from head_detector import HeadDetector

# Inicializar el detector
detector = HeadDetector()

# Directorio de entrada
input_dir = "/home/jocareher/Documents/baby_face_72/images"

# Directorio de salida
output_dir = "/home/jocareher/Documents/vgg_results/labels"
os.makedirs(output_dir, exist_ok=True)

# Iterar sobre los archivos en el directorio
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)
    
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Obtener predicciones y procesar
        predictions = detector(input_path)
        
        # Crear archivo .txt para la imagen actual
        output_txt_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".txt")
        with open(output_txt_path, "w") as file:
            for head in predictions.heads:
                vertices = head.vertices_3d  # Coordenadas de los landmarks procesadas dentro de _parse_predictions
                flattened = vertices.flatten().astype(int)  # Asegurarse de que sean enteros
                file.write(" ".join(map(str, flattened)) + "\n")

print(f"Archivos .txt guardados en: {output_dir}")

Archivos .txt guardados en: /home/jocareher/Documents/vgg_results/labels


In [9]:
def plot_keypoints(image: np.ndarray, keypoints: List[Tuple[float, float]], output_path: str, 
                color: Tuple[int, int, int], min_radius: int = 8, max_radius: int = 16) -> None:
    """
    Plot keypoints on the image and save the result. The size of the keypoints adapts to the image resolution.

    Args:
        image (np.ndarray): Input image.
        keypoints (List[Tuple[float, float]]): List of (x, y) coordinates of keypoints.
        output_path (str): Path to save the plotted image.
        color (Tuple[int, int, int]): Color of the keypoints (B, G, R).
        min_radius (int): Minimum radius for keypoints.
        max_radius (int): Maximum radius for keypoints.
    """
    # Extract image dimensions (height, width, and channels)
    height, width, _ = image.shape
    # Calculate the diagonal length of the image to determine the proportional keypoint size
    image_diagonal = (width**2 + height**2) ** 0.5
    # Calculate radius based on 1% of the diagonal, constrained by min_radius and max_radius
    radius = max(min_radius, min(int(image_diagonal * 0.01), max_radius))
    # Thickness for the circles (-1 means filled circles)
    thickness = -1

    # Iterate over each keypoint (x, y) pair
    for x, y in keypoints:
        # Ensure x and y are integers for OpenCV functions
        x, y = int(x), int(y)
        # Check if the keypoint is within the image boundaries
        if 0 <= x < width and 0 <= y < height:
            # Draw a circle at the keypoint location on the image
            cv2.circle(image, (x, y), radius, color, thickness)

    # Save the image with the plotted keypoints
    cv2.imwrite(output_path, image)


In [10]:
def process_images_and_labels(image_dir: str, label_dir: str, output_dir: str, color: Tuple[int, int, int] = (0, 255, 0)) -> None:
    """
    Process images and labels, draw keypoints on images, and save the results.

    Args:
        image_dir (str): Directory containing the images.
        label_dir (str): Directory containing the label files (.txt).
        output_dir (str): Directory to save the processed images.
        color (Tuple[int, int, int]): Color of the keypoints (B, G, R).
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all files in the image directory
    for image_filename in os.listdir(image_dir):
        if image_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full path to the image file
            image_path = os.path.join(image_dir, image_filename)
            # Construct the corresponding label file path
            label_filename = os.path.splitext(image_filename)[0] + ".txt"
            label_path = os.path.join(label_dir, label_filename)

            # Check if the label file exists
            if os.path.exists(label_path):
                # Read the keypoints from the label file
                keypoints = load_landmarks(label_path)

                # Skip if no valid keypoints are found
                if not keypoints:
                    print(f"[INFO] No valid landmarks for image: {image_filename}. Skipping.")
                    continue

                # Read the image
                image = cv2.imread(image_path)

                # Plot the keypoints on the image
                output_image_path = os.path.join(output_dir, image_filename)
                plot_keypoints(image, keypoints, output_image_path, color)

                print(f"Processed and saved: {output_image_path}")
            else:
                print(f"[WARNING] Label file not found for image: {image_filename}")


In [11]:
# Example usage
image_dir = "/home/jocareher/Documents/baby_face_72/images"
label_dir = "/home/jocareher/Documents/vgg_results/labels"
output_dir = "/home/jocareher/Documents/vgg_results/images"
process_images_and_labels(image_dir, label_dir, output_dir)

Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_197.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_233.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_202.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_313.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_254.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_238.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_262.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_140.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_222.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_340.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_27.JPG
Processed and saved: /home/jocareher/Documents/vgg_results/images/face_bcn_132.JPG
Proce